## Preparation: Data

The time series data for each zone’s prices are sourced from the ENTSO-E Transparency
Platform (TP), an online data repository for European electricity system information established
under Regulation (EU) No. 543/2013, known as the “Transparency Regulation”.

Python client for the ENTSO-E API (european network of transmission system operators for electricity) is used to source the data.
Documentation of the API found on https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html

In [1]:
# Import packages
import pandas as pd
from entsoe import EntsoePandasClient, Area
import numpy as np

In [2]:
#Register client with api Key
api_key="..."
client = EntsoePandasClient(api_key=api_key)

Bidding Zones are identified using the EIC (Energy Identification Code). The EIC is a standardized system for uniquely identifying various entities involved in the energy market. Developed and maintained by the European Network of Transmission System Operators for Electricity (ENTSO-E), the EIC system helps in ensuring consistent and unambiguous identification of participants and locations within the energy sector across Europe.

In [1]:
# Domain mappings
DOMAIN_MAPPINGS = {
    
    # 'AL': '10YAL-KESH-----5', # no values on TP
    'AT': '10YAT-APG------L', # from 30.09.2018 22:00
    'BE': '10YBE----------2', # everything runs
    # 'BA': '10YBA-JPCC-----D', # no values on TP
    #'BG': '10YCA-BULGARIA-R', # from 31.10.2016 22:00
    #'HR': '10YHR-HEP------M', # from 17.10.2017 22:00
    # 'CY': '10YCY-1001A0003J', # no values on TP
    'CZ': '10YCZ-CEPS-----N', # everything runs
    
    #'DK': '10Y1001A1001A65H', see below
    
    'EE': '10Y1001A1001A39I', # everything runs
    'FI': '10YFI-1--------U', # everything runs
    'FR': '10YFR-RTE------C', # everything runs
    # 'GE': '10Y1001A1001B012', # no values on TP
    
    #'DE': '10Y1001A1001A83F', 
    # Germany formed a bidding zone with AT and LU until 30.09.2018 21:00.
    'DE-AT-LU': '10Y1001A1001A63L', # everything runs
    # From 30.09.2018 22:00, the bidding zone consists only of DE and LU. AT has separated, see above!
    'DE-LU': '10Y1001A1001A82H', # everything runs
    
    
    'GR': '10YGR-HTSO-----Y', # everything runs
    'HU': '10YHU-MAVIR----U', # everything runs
    #'IE': '10Y1001A1001A59C', # from 30.09.2018 22:00
    
     #'IT': '10YIT-GRTN-----B', see below
    
    # 'XK': '10Y1001C--00100H', # no values on TP
    'LV': '10YLV-1001A00074', # everything runs
    'LT': '10YLT-1001A0008Q', # everything runs
    
    #'LU': '10YLU-CEGEDEL-NQ',
    # Luxembourg formed a bidding zone with AT and DE until 30.09.2018 21:00.
    # From 30.09.2018 22:00, the bidding zone consists only of DE and LU. AT has separated!
    
    # 'MT': '10Y1001A1001A93C', # no values on TP
    # 'MD': '10Y1001A1001A990', # no values on TP
    #'ME': '10YCS-CG-TSO---S', # from 26.04.2018 22:00
    'NL': '10YNL----------L', # everything runs
    #'MK': '10YMK-MEPSO----8', # from 10.05.2023 22:00
    
    #'NO': '10YNO-0--------C', see below
    
    'PL': '10YPL-AREA-----S', # currency conversion necessary 
    'PT': '10YPT-REN------W', # everything runs
    #'RO': '10YRO-TEL------P', # everything runs, too many outliers
    #'RS': '10YCS-SERBIATSOV', # from 22.12.2016 23:00
    'SK': '10YSK-SEPS-----K', # everything runs
    'SI': '10YSI-ELES-----O', # everything runs
    'ES': '10YES-REE------0', # everything runs
    
     #'SE': '10YSE-1--------K', see below
    
    'CH': '10YCH-SWISSGRIDZ', # everything runs
    # 'TR': '10YTR-TEIAS----W', # no values on TP
    
    #'UA': '10YUA-WEPS-----0', see below

    #'GB': '10YGB----------A', # Data available till 31.12.2020
    #'GB-NIR': '10Y1001A1001A016', # no values on TP
    
    # Countries consisting of multiple bidding zones
    
    'IT-CALABRIA': '10Y1001C--00096J',
    'IT-CNOR': '10Y1001A1001A70O',
    'IT-CSUD': '10Y1001A1001A71M',
    'IT-GR': '10Y1001A1001A66F', # IT-GR BZ
    'IT-NORD': '10Y1001A1001A73I',
    'IT-NORD-AT': '10Y1001A1001A80L',
    'IT-NORD-CH': '10Y1001A1001A68B',
    'IT-NORD-FR': '10Y1001A1001A81J',
    'IT-NORD-SI': '10Y1001A1001A67D',
    'IT-FOGN': '10Y1001A1001A72K', # IT-Foggia BZ
    'IT-ROSN': '10Y1001A1001A77A', # IT-Rossano BZ
    'IT-BRNN': '10Y1001A1001A699', # IT-Brindisi BZ
    'IT-PRGP': '10Y1001A1001A76C', # IT-Priolo BZ
    'IT-SACOAC': '10Y1001A1001A885',
    'IT-SACODC': '10Y1001A1001A893',
    'IT-SARD': '10Y1001A1001A74G',
    'IT-SICI': '10Y1001A1001A75E',
    'IT-SUD': '10Y1001A1001A788',
    
    
    'NO-1': '10YNO-1--------2',
    'NO-2': '10YNO-2--------T',
    'NO-2NSL': '50Y0JVU59B4JWQCU', # virtual bidding zone
    'NO-3': '10YNO-3--------J',
    'NO-4': '10YNO-4--------9',
    'NO-5': '10Y1001A1001A48H',
    
    'SE-1': '10Y1001A1001A44P',
    'SE-2': '10Y1001A1001A45N',
    'SE-3': '10Y1001A1001A46L',
    'SE-4': '10Y1001A1001A47J',
    
    'DK-1': '10YDK-1--------W',
    'DK-2': '10YDK-2--------M',
    
    #'UA-BEI': '10YUA-WEPS-----0', incomplete
    #'UA-IPS': '10Y1001C--000182' incomplete
    
}

## Full data
First the data of all countries is aggregated and saved into a dataframe

In [4]:
#Set year
year = 2016
#Start time
start = pd.Timestamp(str(year)+'-04-29-T00', tz='UTC')
#A few hours from the next year are needed, because the API does not work properly for UTC+1
end = pd.Timestamp(str(year+8)+'-01-01-T00', tz='UTC')

In [5]:
all_data=pd.DataFrame()
countries_success=[]
countries_missing=[]

In [ ]:
for country_code in DOMAIN_MAPPINGS:
    mapping = DOMAIN_MAPPINGS[country_code]
    try:
        # Get data for day-ahead prices for a single country
        day_ahead_prices = client.query_day_ahead_prices(mapping, start=start, end=end)
        day_ahead_prices = day_ahead_prices.drop(day_ahead_prices[day_ahead_prices.index.minute != 00].index)
        day_ahead_prices = day_ahead_prices.tz_convert("UTC")
        #day_ahead_prices = day_ahead_prices.where(day_ahead_prices.index.year == year).dropna()

    except Exception as error:
        print(error)
        print("Country excluded: " + country_code)
        countries_missing.append(country_code)
    else:
        # Create a DataFrame for day-ahead prices for this country
        data_one_country = pd.DataFrame(day_ahead_prices)
        data_one_country = data_one_country.rename(columns={0: "Day_ahead_price"})
        data_one_country.insert(loc=0, column='Country', value=country_code)
        data_one_country.insert(loc=0, column='Year', value=data_one_country.index.year)
        #data_one_country.insert(loc=0, column='Index', value=data_one_country.index)

        # Add day-ahead price data to the final DataFrame
        all_data = pd.concat([all_data, data_one_country])

        # Add the country code to the list of successful countries
        countries_success.append(country_code)

print(countries_success)


### Add single country
If a single country is missing due to a connection error, it can be appended

In [7]:
# Specify das fehlende Land
country_code = "NO-1"

# Daten werden abgerufen und das Land wird hinzugefügt
if country_code is not None:
    mapping = DOMAIN_MAPPINGS[country_code]

    try:
        # Daten für ein einzelnes Land abrufen
        # Nur Day-ahead-Preise abrufen
        day_ahead_prices = client.query_day_ahead_prices(mapping, start=start, end=end)
        day_ahead_prices = day_ahead_prices.drop(day_ahead_prices[day_ahead_prices.index.minute != 0].index)
        day_ahead_prices = day_ahead_prices.tz_convert("UTC")
        #day_ahead_prices = day_ahead_prices.where(day_ahead_prices.index.year == year).dropna()

        # Daten für Day-ahead-Preise in einen DataFrame einfügen
        data_one_country = pd.DataFrame(day_ahead_prices)
        data_one_country = data_one_country.rename(columns={0: "Day_ahead_price"})
        data_one_country.insert(loc=0, column='Country', value=country_code)
        data_one_country.insert(loc=0, column='Year', value=data_one_country.index.year)
        #data_one_country.insert(loc=0, column='Index', value=data_one_country.index)

        # Day-ahead-Preisdaten dem Gesamtdaten-DataFrame hinzufügen
        all_data = pd.concat([all_data, data_one_country])

        # Das Land zur Liste der erfolgreich abgerufenen Länder hinzufügen
        countries_success.append(country_code)

    except Exception as error:
        print(error)
        print("Country excluded: " + country_code)
        countries_missing.append(country_code)


In [ ]:
all_data.to_csv('Data.csv', index=True)